In [1]:
# load dataset data/test.csv and data/train.csv

import pandas as pd

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print(train.shape, test.shape)

train.head()

(120000, 2) (7600, 2)


,text,label
0,wall st bear claw back black reuters reuters s...,2
1,carlyle look toward commercial aerospace reute...,2
2,oil economy cloud stock outlook reuters reuter...,2
3,iraq halt oil export main southern pipeline re...,2
4,oil price soar alltime record posing new menac...,2


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize the text data
tfidf = TfidfVectorizer(stop_words='english')

# Learn vocabulary from training texts and vectorize training texts.
tfidf.fit(train.text)

# transform the training data
train_tfidf = tfidf.transform(train.text)

# transform the test data
test_tfidf = tfidf.transform(test.text)

print(train_tfidf.shape, test_tfidf.shape)

(120000, 77346) (7600, 77346)


In [3]:
# simply use the vectorized data as input to the model
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(train_tfidf, train.label)

/data/tianyang/anaconda3/envs/repobench/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [4]:
#  predict and evaluate the model
from sklearn.metrics import classification_report

# predict on the test set
predictions = classifier.predict(test_tfidf)

# calculate the classification report
print(classification_report(test.label, predictions, digits=4))

              precision    recall  f1-score   support

           0     0.9353    0.9053    0.9200      1900
           1     0.9563    0.9800    0.9680      1900
           2     0.8780    0.8789    0.8785      1900
           3     0.8844    0.8900    0.8872      1900

    accuracy                         0.9136      7600
   macro avg     0.9135    0.9136    0.9134      7600
weighted avg     0.9135    0.9136    0.9134      7600



In [5]:
# svm classifier
from sklearn.svm import LinearSVC

classifier = LinearSVC()
classifier.fit(train_tfidf, train.label)

/data/tianyang/anaconda3/envs/repobench/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LinearSVC()

In [7]:
# predict on the test set
predictions = classifier.predict(test_tfidf)

# calculate the classification report
print(classification_report(test.label, predictions, digits=4))

              precision    recall  f1-score   support

           0     0.9368    0.9047    0.9205      1900
           1     0.9573    0.9800    0.9685      1900
           2     0.8864    0.8874    0.8869      1900
           3     0.8900    0.8984    0.8942      1900

    accuracy                         0.9176      7600
   macro avg     0.9176    0.9176    0.9175      7600
weighted avg     0.9176    0.9176    0.9175      7600



In [8]:
# naive bayes classifier
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(train_tfidf, train.label)

MultinomialNB()

In [9]:
# predict on the test set
predictions = classifier.predict(test_tfidf)

# calculate the classification report
print(classification_report(test.label, predictions, digits=4))

              precision    recall  f1-score   support

           0     0.9150    0.8947    0.9047      1900
           1     0.9508    0.9763    0.9634      1900
           2     0.8590    0.8626    0.8608      1900
           3     0.8800    0.8721    0.8760      1900

    accuracy                         0.9014      7600
   macro avg     0.9012    0.9014    0.9012      7600
weighted avg     0.9012    0.9014    0.9012      7600



In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '4'

# Convert sparse matrix to PyTorch tensor
def sparse_to_tensor(sparse_matrix):
    sparse_matrix = sparse_matrix.tocoo()
    values = sparse_matrix.data
    indices = np.vstack((sparse_matrix.row, sparse_matrix.col))
    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = sparse_matrix.shape
    return torch.sparse.FloatTensor(i, v, torch.Size(shape)).to_dense()

train_features = sparse_to_tensor(train_tfidf)
train_labels = torch.LongTensor(train.label)

test_features = sparse_to_tensor(test_tfidf)
test_labels = torch.LongTensor(test.label)

class MyDataset(Dataset):
    def __init__(self, feature_vectors, labels):
        self.labels = labels
        self.feature_vectors = feature_vectors

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.feature_vectors[idx], self.labels[idx]

# Create dataset objects
train_dataset = MyDataset(train_features, train_labels)
test_dataset = MyDataset(test_features, test_labels) 

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Model definition
class Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Hyperparameters
input_dim = len(tfidf.get_feature_names_out())
hidden_dim = 32
output_dim = 4  # Assuming 4 classes
epochs = 10

# Model instantiation
model = Model(input_dim, hidden_dim, output_dim)
# Move model to GPU if CUDA is available
model.cuda()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
for epoch in range(epochs):
    model.train()
    for i, (features, labels) in tqdm(enumerate(train_loader)):
        # move tensors to GPU if CUDA is available
        features = features.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    # Evaluation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for features, labels in tqdm(test_loader):
            # move tensors to GPU if CUDA is available
            features = features.cuda()
            labels = labels.cuda()
            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{epochs}], Accuxracy: {accuracy:.2f}%')

print("Training complete")

1it [00:01,  1.88s/it]

Epoch [1/10], Step [1/938], Loss: 1.3839


101it [03:02,  1.78s/it]

Epoch [1/10], Step [101/938], Loss: 1.3796


201it [06:06,  1.82s/it]

...

Epoch [10/10], Step [801/938], Loss: 0.1901


901it [28:17,  2.07s/it]

Epoch [10/10], Step [901/938], Loss: 0.1652


938it [29:24,  1.88s/it]
100%|██████████| 60/60 [01:53<00:00,  1.89s/it]

Epoch [10/10], Accuxracy: 91.72%
Training complete


In [ ]:
# predict on the test set
predictions = classifier.predict(test_tfidf)

# calculate the classification report
print(classification_report(test.label, predictions))